In [65]:
import sys

sys.path.append('../../code/')
import os
import json
from datetime import datetime
import time
from math import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

import igraph as ig

from collections import *

from load_data import load_citation_network_igraph, case_info

%load_ext autoreload
%autoreload 2
%matplotlib inline

data_dir = '../../data/'
court_name = 'scotus'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
#takes a numpy adjacency matrix and returns the pagerank of the vertices
#all variable names follow along MLPP 17.34
#please note: the eigen vector that is returned is normalized, this is not the case for igraph.pagerank(),
#but this pagerank is proportional to igraph's pagerank
def PageRank(A):
    #the adjacency matrix used by MLPP is defined as Gij = 1 iff j points to i. This requires the transposed A.
    G = A.getT()
    n = A.shape[0]
    p = 0.85
    delta = (1.0-p)/n

    #vector which represents the outdegree of each vertex
    c = np.matrix(np.zeros((n,1)))
    for j in range(0,n):
        outdegree = np.sum(G[:,j])
        c[j] = outdegree

    #below these variable represent the decomposition of the transition matrix M
    D = np.matrix(np.zeros((n, n)))
    for j in range(0,n):
        if not c[j] == 0:
            D[j,j] = 1.0/c[j]
        else:
            D[j,j] = 0
 
    z = np.matrix(np.zeros((n,1)))
    for j in range(0,n):
        if not c[j] == 0:
            z[j] = delta
        else:
            z[j] = 1.0/n
    zT = z.getT()

    #variables are combined to produce M
    M = p*G*D + 1*zT

    #Power method to derive the first eigen-vector
    v = np.matrix(np.ones((n,1)))
    v = v/np.linalg.norm(v)
    iterations = 0
    converged = False
    while not converged:
        iterations += 1
        tmp = M*v
        norm = np.linalg.norm(tmp)
        new_v = tmp/norm
        
        difference = np.absolute(np.sum(v-new_v))
        if difference < 0.00001:
            converged = True
        
        v = new_v
        
    return v

PageRank on simple 5x5 test graph

In [67]:
#create test adjacency matrix
node_names = ['A', 'B', 'C', 'D', 'E']
a = pd.DataFrame([[0,1,1,0,1],[0,0,1,0,1],[0,0,0,1,0],[0,0,0,0,1],[0,0,0,0,0]],
    index=node_names, columns=node_names)
a_numpy = np.matrix(a)
print a_numpy

[[0 1 1 0 1]
 [0 0 1 0 1]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 0 0 0]]


In [68]:
#find pagerank using igraph (to check our work)
g = ig.Graph.Adjacency(a_numpy.tolist())
ig_pagerank = g.pagerank()
print ig_pagerank

[0.09375108949020011, 0.12031389817909016, 0.1714473049052035, 0.23948129865962306, 0.3750064087658831]


In [69]:
#find pagerank using our implementation
pagerank = PageRank(a_numpy)
print pagerank

[[ 0.18726233]
 [ 0.24031935]
 [ 0.34245496]
 [ 0.47835126]
 [ 0.74905548]]


PageRank on portion of SCOTUS network (before 1900)

In [70]:
G = load_citation_network_igraph(data_dir, court_name)

print 'loaded %s network with %d cases and %d edges' % (court_name, len(G.vs), len(G.es))

0 seconds for 250465 edges
loaded scotus network with 33248 cases and 250465 edges


In [71]:
sub_vs = G.vs.select(year_lt=1900)
sub_G = G.subgraph(sub_vs)
sub_G.summary()

'IGRAPH DN-- 10446 25674 -- \n+ attr: court (v), name (v), year (v)'

In [72]:
scotus_adjacency = sub_G.get_adjacency()

In [75]:
#find pagerank using igraph (to check our work)
print sub_G.pagerank()/np.linalg.norm(sub_G.pagerank())

[ 0.00516806  0.00503505  0.01066494 ...,  0.00396808  0.00396808
  0.00396808]


In [74]:
#find pagerank using our implementation
print PageRank(np.matrix(scotus_adjacency.data))

[[ 0.00516807]
 [ 0.00503505]
 [ 0.01066495]
 ..., 
 [ 0.00396808]
 [ 0.00396808]
 [ 0.00396808]]
